# 0-Gaussian model 
## p(xi​)=k=1∑K​πk​N(xi​∣μk​,Σk​)

## 1-importing libraries 

In [3]:
import numpy as np

## 2- 
If Σ is not invertible, np.linalg.inv() will crash.(too few points )

eps to the diagonal of Σ to push eigen values from zero makes sure Σ is invertible (no sing)

## Multivarite gaussian denisty: how like is the given point under the cluster 

## $\mathcal{N}(x \mid \mu, \Sigma) = \frac{1}{(2\pi)^{d/2} |\Sigma|^{1/2}} \exp\left(-\frac{1}{2} (x-\mu)^T \Sigma^{-1} (x-\mu)\right)$


In [4]:
def gaussian_pdf(X, mean, cov):
    """
    Computes multivariate Gaussian probability density function for all data points.

    Parameters:
    X    : (N, D) data matrix
    mean : (D,) mean vector
    cov  : (D, D) covariance matrix

    Returns:
    pdf  : (N,) probability values
    """
    N, D = X.shape
    
    # Add small value to diagonal for numerical stability
    eps = 1e-6
    cov = cov + eps * np.eye(D)
    
    # Compute determinant and inverse
    det = np.linalg.det(cov)
    inv = np.linalg.inv(cov)
    
    # Normalization constant
    norm_const = 1.0 / np.sqrt((2 * np.pi) ** D * det)
    
    # Difference from mean
    diff = X - mean
    
    # Exponent term (Mahalanobis distance)
    exponent = -0.5 * np.sum(diff @ inv * diff, axis=1)
    
    return norm_const * np.exp(exponent)


## 3-Intializing the parameters
## Mean :The center cluster (μk) : starting with random points
## Covariance :(Σk) shape depending on the type(full, tied, diagonal, spherical)
## Mixing(πk): Gaussian weight intially all = .k since we have no prior knowlege EM will update it 

In [5]:
def initialize_gmm(X, K, covariance_type):
    """
    Initializes GMM parameters.

    Returns:
    pi    : (K,) mixing coefficients
    mu    : (K, D) means
    sigma : covariance parameters (depends on type)
    """
    N, D = X.shape
    
    # Mixing coefficients (equal initially)
    pi = np.ones(K) / K
    
    # Randomly choose data points as initial means
    indices = np.random.choice(N, K, replace=False)
    mu = X[indices]
    
    # Initialize covariance
    if covariance_type == "full":
        sigma = np.array([np.cov(X.T) for _ in range(K)])
        # each cluster can stretch and rotate  ELLIPSE
    elif covariance_type == "tied":
        sigma = np.cov(X.T)
        # each cluster has its own shape ELLIPSE same mean
    elif covariance_type == "diag": 
        sigma = np.array([np.var(X, axis=0) for _ in range(K)])
        ## RECTANGLE (NO ROTATION)
    elif covariance_type == "spherical":
        sigma = np.array([np.var(X) for _ in range(K)])
        ##CIRCLE (NO ROTATION TOO)
    else:
        raise ValueError("Invalid covariance type")
    
    return pi, mu, sigma


## 4 E-Step: Computing responsibilities(probability of belonging to each cluster)
$$
\gamma_{ik} = \frac{\pi_k \, \mathcal{N}(x_i \mid \mu_k, \Sigma_k)}
{\sum_{j=1}^{K} \pi_j \, \mathcal{N}(x_i \mid \mu_j, \Sigma_j)} 
$$

Where:  
- $\gamma_{ik}$ = probability that point $x_i$ belongs to cluster $k$  
- $\pi_k$ = mixing coefficient for cluster $k$  
- $\mathcal{N}(x_i \mid \mu_k, \Sigma_k)$ = multivariate Gaussian PDF  
- $K$ = total number of clusters

In [6]:
def e_step(X, pi, mu, sigma, covariance_type):
    """
    E-step: compute responsibilities (soft membership probabilities) for all points.

    Parameters:
    X : (N, D) data matrix
    pi : (K,) mixing coefficients
    mu : (K, D) mean vectors
    sigma : covariance parameters
    covariance_type : 'full', 'tied', 'diag', or 'spherical'

    Returns:
    gamma : (N, K) responsibility matrix
    """
    N, D = X.shape
    K = len(pi)
    gamma = np.zeros((N, K))
    
    # Loop over clusters
    for k in range(K):
        # Select correct covariance for this type
        if covariance_type == "full":
            cov = sigma[k]
        elif covariance_type == "tied":
            cov = sigma
##for all data since all use same covariance matrix only mean differ 
        elif covariance_type == "diag":
            cov = np.diag(sigma[k])
##vector of variance along each feature 

        elif covariance_type == "spherical":
            cov = np.eye(D) * sigma[k]
##identity matrix scaled by variance
        else:
            raise ValueError("Invalid covariance_type")
        
        # Compute probability of all points under this Gaussian
        gamma[:, k] = pi[k] * gaussian_pdf(X, mu[k], cov)
    
    # Normalize across clusters to get probabilities
    gamma /= gamma.sum(axis=1, keepdims=True)
    
    return gamma


## 5 M-step updating the parameters
# Gaussian Mixture Model (GMM) Equations - M-step

### Effective number of points per cluster:
$$
N_k = \sum_{i=1}^{N} \gamma_{ik}
$$

### Mixing coefficients:
$$
\pi_k = \frac{N_k}{N}
$$

### Means:
$$
\mu_k = \frac{1}{N_k} \sum_{i=1}^{N} \gamma_{ik} x_i
$$

### Covariances:

**Full covariance (general case):**
$$
\Sigma_k = \frac{1}{N_k} \sum_{i=1}^{N} \gamma_{ik} (x_i - \mu_k)(x_i - \mu_k)^T
$$

**Tied covariance (shared across clusters):**
$$
\Sigma_{\text{tied}} = \frac{1}{N} \sum_{k=1}^{K} \sum_{i=1}^{N} \gamma_{ik} (x_i - \mu_k)(x_i - \mu_k)^T
$$

**Diagonal covariance (no correlation, axes-aligned):**
$$
\Sigma_k^{\text{diag}} = \text{diag} \Bigg( \frac{1}{N_k} \sum_{i=1}^{N} \gamma_{ik} (x_i - \mu_k)^2 \Bigg)
$$

**Spherical covariance (same variance in all directions):**
$$
\sigma_k^{\text{spherical}} = \frac{1}{D N_k} \sum_{i=1}^{N} \gamma_{ik} \|x_i - \mu_k\|^2
$$



In [7]:
def m_step(X, gamma, covariance_type):
    """
    M-step: update GMM parameters based on responsibilities.

    Parameters:
    X : (N, D) data matrix
    gamma : (N, K) responsibilities
    covariance_type : 'full', 'tied', 'diag', 'spherical'

    Returns:
    pi : updated mixing coefficients
    mu : updated means
    sigma : updated covariances
    """
    N, D = X.shape
    K = gamma.shape[1]
    
    # Effective number of points per cluster
    Nk = gamma.sum(axis=0)
    
    # Update mixing coefficients
    pi = Nk / N
    
    # Update means
    mu = (gamma.T @ X) / Nk[:, None]
    
    # Update covariances
    if covariance_type == "full":
        sigma = np.zeros((K, D, D))
        for k in range(K):
            diff = X - mu[k]
            sigma[k] = (gamma[:, k][:, None] * diff).T @ diff / Nk[k]
    
    elif covariance_type == "tied":
        sigma = np.zeros((D, D))
        for k in range(K):
            diff = X - mu[k]
            sigma += (gamma[:, k][:, None] * diff).T @ diff
        sigma /= N
    
    elif covariance_type == "diag":
        sigma = np.zeros((K, D))
        for k in range(K):
            diff = X - mu[k]
            sigma[k] = (gamma[:, k][:, None] * diff**2).sum(axis=0) / Nk[k]
    
    elif covariance_type == "spherical":
        sigma = np.zeros(K)
        for k in range(K):
            diff = X - mu[k]
            sigma[k] = (gamma[:, k] * np.sum(diff**2, axis=1)).sum() / (Nk[k] * D)
    
    return pi, mu, sigma


## 6-log likelihood(how well current parameters fit data)
$$
\log L = \sum_{i=1}^{N} \log \Bigg( \sum_{k=1}^{K} \pi_k \, \mathcal{N}(x_i \mid \mu_k, \Sigma_k) \Bigg)
$$

In [8]:
def compute_log_likelihood(X, pi, mu, sigma, covariance_type):
    """
    Compute total log-likelihood of the data under current GMM parameters.
    """
    N, D = X.shape
    K = len(pi)
    likelihood = np.zeros((N, K))
    
    for k in range(K):
        if covariance_type == "full":
            cov = sigma[k]
        elif covariance_type == "tied":
            cov = sigma
        elif covariance_type == "diag":
            cov = np.diag(sigma[k])
        elif covariance_type == "spherical":
            cov = np.eye(D) * sigma[k]
        likelihood[:, k] = pi[k] * gaussian_pdf(X, mu[k], cov)
    
    log_likelihood = np.sum(np.log(likelihood.sum(axis=1)))
    return log_likelihood

## 6 : Full EM Algorithm 
## E-step → M-step → Log-likelihood check reapeat

In [9]:
def gmm_fit(X, K, covariance_type="full", max_iter=100, tol=1e-4):
    """
    Fits a Gaussian Mixture Model to data using EM algorithm.

    Returns:
    pi : mixing coefficients
    mu : cluster means
    sigma : cluster covariances
    gamma : responsibilities
    log_likelihoods : list of log-likelihoods per iteration
    """
    # Initialize parameters
    pi, mu, sigma = initialize_gmm(X, K, covariance_type)
    log_likelihoods = []
    
    for iteration in range(max_iter):
        # E-step: compute responsibilities
        gamma = e_step(X, pi, mu, sigma, covariance_type)
        
        # M-step: update parameters
        pi, mu, sigma = m_step(X, gamma, covariance_type)
        
        # Log-likelihood
        ll = compute_log_likelihood(X, pi, mu, sigma, covariance_type)
        log_likelihoods.append(ll)
        
        # Check convergence
        if iteration > 0 and abs(log_likelihoods[-1] - log_likelihoods[-2]) < tol:
            print(f"Converged at iteration {iteration}")
            break
    
    return pi, mu, sigma, gamma, log_likelihoods
